In [ ]:
try:
    import timm
except:
    !pip install /kaggle/input/notebook937eeb7d5c/timm-0.6.12-py3-none-any.whl
try:
    import gdcm
except:
    !pip install /kaggle/input/rsna-2022-whl/python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
try:
    import pylibjpeg
except:
    !pip install /kaggle/input/rsna-2022-whl/{pydicom-2.3.0-py3-none-any.whl,pylibjpeg-1.4.0-py3-none-any.whl,python_gdcm-3.0.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl}
!pip install -q /kaggle/input/rsna-bcd-whl-ds/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl

In [ ]:
import gc
import torch
import torch.nn as nn
from torch.nn import functional as F
from PIL import Image
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import OneCycleLR
from sklearn.model_selection import GroupKFold 
from sklearn.metrics import f1_score,accuracy_score
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
from tqdm import tqdm
import glob
import shutil
import timm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import dicomsdl
from joblib import Parallel, delayed
import random
import warnings
warnings.filterwarnings("ignore")

In [ ]:
test_images = glob.glob("/kaggle/input/rsna-breast-cancer-detection/test_images/*/*.dcm")

In [ ]:
test_images

In [ ]:
save_path = "/kaggle/temp/"
INPUT_SIZE = 1280
DEVICE = 'cuda'
os.makedirs(save_path, exist_ok = True)

In [ ]:
os.listdir(save_path)

In [ ]:
df = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
df

In [ ]:
'''def load_image_dicomsdl(img_path):
    dataset = dicomsdl.open(img_path)
    img = dataset.pixelData()

    try:
            # Load only the variables we need
            center = dataset["WindowCenter"]
            width = dataset["WindowWidth"]
            bits_stored = dataset["BitsStored"]
            voi_lut_function = dataset["VOILUTFunction"]

            # For sigmoid it's a list, otherwise a single value
            if isinstance(center, list):
                center = center[0]
            if isinstance(width, list):
                width = width[0]

            # Set y_min, max & range
            y_min = 0
            y_max = float(2**bits_stored - 1)
            y_range = y_max

            # Function with default LINEAR (so for Nan, it will use linear)
            if voi_lut_function == "SIGMOID":
                img = y_range / (1 + np.exp(-4 * (img - center) / width)) + y_min
            else:
                # Checks width for < 1 (in our case not necessary, always >= 750)
                center -= 0.5
                width -= 1

                below = img <= (center - width / 2)
                above = img > (center + width / 2)
                between = np.logical_and(~below, ~above)

                img[below] = y_min
                img[above] = y_max
                if between.any():
                    img[between] = (
                        ((img[between] - center) / width + 0.5) * y_range + y_min
                    )
    except Exception as e:
#         dataset = dicoml.open(img_path)
        img = dataset.pixelData()


    img = (img - img.min()) / (img.max() - img.min())

    if dataset["PhotometricInterpretation"] == "MONOCHROME1":
        img = 1 - img

    img = (img * 255).astype(np.uint8)

    return img'''

In [ ]:
def img2roi(img):
    # Binarize the image
    bin_img = cv2.threshold(img, 20, 255, cv2.THRESH_BINARY)[1]

    # Make contours around the binarized image, keep only the largest contour
    contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contour = max(contours, key=cv2.contourArea)

    # Find ROI from largest contour
    ys = contour.squeeze()[:, 0]
    xs = contour.squeeze()[:, 1]
    roi =  img[np.min(xs):np.max(xs), np.min(ys):np.max(ys)]
    
    return roi

In [ ]:
def process(path, size):
    patient = path.split('/')[-2]
    image = path.split('/')[-1][:-4]
    dicom = dicomsdl.open(path)
    img = dicom.pixelData(storedvalue=False)
    img = (img - img.min()) / (img.max()-img.min())
    if dicom.PhotometricInterpretation == "MONOCHROME1":
        img = 1.0- img
    img = (img*255).astype(np.uint8)
    img = img2roi(img)
    final_img = Image.fromarray(img)
    final_img = final_img.resize((int(INPUT_SIZE/2),int(INPUT_SIZE)),Image.Resampling.LANCZOS)
    final_img.save(save_path+f"{patient}_{image}.png")

In [ ]:
'''def process(path, size):
    patient = path.split('/')[-2]
    image = path.split('/')[-1][:-4]
    img = load_image_dicomsdl(path)
    final_img = Image.fromarray(img)
    final_img = final_img.resize((int(INPUT_SIZE/2),int(INPUT_SIZE)),Image.Resampling.LANCZOS)
    final_img.save(save_path+f"{patient}_{image}.png")'''

In [ ]:
_ = Parallel(n_jobs=4)(
    delayed(process)(uid, size=INPUT_SIZE)
    for uid in tqdm(test_images)
)

In [ ]:
os.listdir(save_path)

In [ ]:
'''from matplotlib import pyplot as plt
import cv2
fig = plt.figure(figsize = (10,10))
plt.subplot(221)
img1 = cv2.imread(save_path+'10008_361203119.png')
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
plt.imshow(img1)
plt.subplot(222)
img2 = cv2.imread(save_path+'10008_736471439.png')
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
plt.imshow(img2)
plt.subplot(223)
img3 = cv2.imread(save_path+'10008_1591370361.png')
img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
plt.imshow(img3)
plt.subplot(224)
img4 = cv2.imread(save_path+'10008_68070693.png')
img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
plt.imshow(img4)
plt.show()'''

In [ ]:
'''import torchvision

def get_transforms(aug=False):

    def transforms(img):
        img = img.convert('RGB')#.resize((512, 512))
        if aug:
            tfm = [
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.RandomRotation(degrees=(-5, 5)), 
                torchvision.transforms.RandomResizedCrop((1024, 512), scale=(0.8, 1), ratio=(0.45, 0.55)) 
            ]
        else:
            tfm = [
                torchvision.transforms.RandomHorizontalFlip(0.5),
                torchvision.transforms.Resize((1024, 512))
            ]
        img = torchvision.transforms.Compose(tfm + [            
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=0.2179, std=0.0529),
            
        ])(img)
        return img

    return lambda img: transforms(img)'''


In [ ]:
from albumentations.pytorch.transforms import ToTensorV2
aug_resize_norm = A.Compose([
    A.Resize(INPUT_SIZE, INPUT_SIZE/2),
    A.Normalize(),
    ToTensorV2(),
])

In [ ]:
import cv2
class BreastCancerDataSet(torch.utils.data.Dataset):
    def __init__(self, df, path, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transforms = transforms

    def __getitem__(self, i):

        path = f'{self.path}/{self.df.iloc[i].patient_id}_{self.df.iloc[i].image_id}.png'
        try:
            #img = Image.open(path).convert('RGB')
            img = cv2.imread(path)
        except Exception as ex:
            print(path, ex)
            return None

        if self.transforms is not None:
            #img = self.transforms(img)
            img = self.transforms(image=img)['image']
        return img

    def __len__(self):
        return len(self.df)

In [ ]:
'''from PIL import Image
class BreastCancerDataSet(torch.utils.data.Dataset):
    def __init__(self, df, path, transforms=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transforms = transforms

    def __getitem__(self, i):

        path = f'{self.path}/{self.df.iloc[i].patient_id}_{self.df.iloc[i].image_id}.png'
        try:
            img = Image.open(path).convert('RGB')
        except Exception as ex:
            print(path, ex)
            return None

        if self.transforms is not None:
            img = self.transforms(img)


        return img

    def __len__(self):
        return len(self.df)'''

In [ ]:
aux_classes = [2,2,6,2,2,2,4,5,2,10,10]

In [ ]:
class BreastCancerModel(torch.nn.Module):
    def __init__(self, aux_classes, model_type, dropout=0.):
        super().__init__()
        self.model = timm.create_model(model_type, pretrained=False, drop_rate=dropout)

        self.backbone_dim = self.model(torch.randn(1, 3, 1280, 640)).shape[-1]

        self.nn_cancer = torch.nn.Sequential(
            torch.nn.Linear(self.backbone_dim, 1),
        )
        self.nn_aux = torch.nn.ModuleList([
            torch.nn.Linear(self.backbone_dim, n) for n in aux_classes
        ])

    def forward(self, x):
        # returns logits
        x = self.model(x)

        cancer = self.nn_cancer(x).squeeze()
        aux = []
        for nn in self.nn_aux:
            aux.append(nn(x).squeeze())
        return cancer, aux

    def predict(self, x):
        cancer, aux = self.forward(x)
        sigaux = []
        for a in aux:
            sigaux.append(torch.softmax(a, dim=-1))
        return torch.sigmoid(cancer), sigaux

In [ ]:
def load_model(path, model=None):
    data = torch.load(path, map_location=DEVICE)
    if model is None:
        model = BreastCancerModel(aux_classes, data['model_type'])
    model.load_state_dict(data['model'])
    return model, data['threshold'], data['model_type']


In [ ]:
models = []
thresholds = []
MODELS_PATH = '/kaggle/input/efficientmodel/'
for fname in tqdm(sorted(os.listdir(MODELS_PATH))):
    model, thres, model_type = load_model(MODELS_PATH+fname)
    model = model.to(DEVICE)
    models.append((model, thres))
    thresholds.append(thres)
    print(f'fname:{fname}, model_type:{model_type}, thres:{thres}')

In [ ]:
use_TTA = True
top3 = False
geomeanpatient = False
geomeanpredict =False

In [ ]:
def models_predict(models, ds, max_batches=1e9):
    dl_test = torch.utils.data.DataLoader(ds, batch_size=16, shuffle=False, num_workers=os.cpu_count())
    for m, thres in models:
        m.eval()

    with torch.no_grad():
        predictions = []
        for idx, X in enumerate(tqdm(dl_test, mininterval=30)):
            pred = torch.zeros(len(X), len(models))
            for idx, (m, thres) in enumerate(models):
                X = X.to(DEVICE)
                pred1 = m.predict(X)[0].squeeze()
                if use_TTA:
                    pred2 = model.predict(torch.flip(X, dims=[-1]))[0].squeeze()
                    preds = (pred1 + pred2) / 2
                else: preds = pred1
                pred[:, idx] = preds.cpu()
            if geomeanpredict:
                predictions.append(pred.prod(dim=-1)**(1/pred.shape[-1]))
            else:
                predictions.append(pred.mean(dim=-1))
            if idx >= max_batches:
                break
        return torch.concat(predictions).numpy()

In [ ]:
thresmax = np.max(thresholds)
thresmean = np.mean(thresholds)

In [ ]:
#ds_test = BreastCancerDataSet(df, save_path, get_transforms(False))
ds_test = BreastCancerDataSet(df, save_path, aug_resize_norm)
models_pred = models_predict(models, ds_test)
models_pred

In [ ]:
df['cancer'] = models_pred


df_sub = df.groupby('prediction_id')[['cancer']].mean()
df_sub

In [ ]:
if top3:
    sup = df.groupby('patient_id')[['cancer']]
    for key, val in sup:
        ids = list(df.loc[df['patient_id'] == key]['prediction_id'])
        top_3 = np.sort(list(df.loc[df['patient_id'] == key]['cancer']))[:-1].mean()
        for idx in ids:
            df_sub.loc[idx, 'cancer'] = top_3

In [ ]:
if geomeanpatient:
    df_sup = df.groupby('prediction_id')
    for key, val in df_sup:
        arr = val['cancer'].to_numpy()
        df_sub.loc[key, 'cancer'] = arr.prod()**(1.0/len(arr))
df_sub

In [ ]:
df_sub['cancer'] = (df_sub.cancer > thresmean).astype(float)
df_sub

In [ ]:
df_sub.to_csv('submission.csv', index=True)
!head submission.csv